In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Librarys

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import callbacks
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(0)
tf.random.set_seed(42)


#  Read & Resize imegs

In [ ]:


file_name = "/kaggle/input/brain-mri-images-for-brain-tumor-detection"

X = []
y = []

yes_path = os.path.join(file_name, "yes")
for img_name in tqdm(os.listdir(yes_path)):
    img_path = os.path.join(yes_path, img_name)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (224, 224))
        X.append(img)
        y.append(1)  

no_path = os.path.join(file_name, "no")
for img_name in tqdm(os.listdir(no_path)):
    img_path = os.path.join(no_path, img_name)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (224, 224))
        X.append(img)
        y.append(0)  

X = np.array(X)
y = np.array(y)

print(f"Total images: {len(X)}, Total labels: {len(y)}")

In [ ]:
#normalization for X
X = X / 255.0 


In [ ]:
# split Data to Train & Test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
# Visualize the training dataset
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(X_train[i])
    plt.title("Tumor" if y_train[i] == 1 else "No Tumor")  
    plt.axis("off")
plt.show()

In [ ]:

X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.int32)   

In [ ]:
# Read Img From Training Data
plt.imshow((X_train[0] * 255).astype("uint8"))  
plt.axis("off")
plt.show()


In [ ]:
from tensorflow.keras.utils import to_categorical

# Convert y_train and y_test to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Batches Of Data

In [ ]:
# Split Data to Batchs
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32) 

# Data Augmentation

In [ ]:
# Layers of Augmentation
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor = 0.15),
        layers.RandomTranslation(height_factor = 0.1, width_factor = 0.1),
        layers.RandomFlip(),
        layers.GaussianNoise(stddev = 0.09),
        layers.RandomContrast(factor = 0.1),
    ],
    name = "img_augmentation"
)

In [ ]:
# Get the class names
class_names = np.unique(y_train)  
print(class_names)

# Show Augmentation effict

In [ ]:
# View the augmentations
plt.figure(figsize=(10, 10))
for image, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        aug_img = img_augmentation(tf.expand_dims(image[0], axis=0))  
        plt.imshow((aug_img[0].numpy() * 255).astype("uint8"))  
        plt.title("Tumor" if np.argmax(labels[0].numpy()) == 1 else "No Tumor")  
        plt.axis("off")
plt.show()


# Bulid the Model

In [ ]:
from keras.applications import vgg16


img_rows, img_cols = 224, 224 


vgg = vgg16.VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg.layers:
    layer.trainable = False
    
# print our layers 
for (i,layer) in enumerate(vgg.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)
    def lw(bottom_model, num_classes):
        """Creates the top or head of the model that will be 
        placed on top of the bottom layers"""

        top_model = bottom_model.output  
        top_model = GlobalAveragePooling2D()(top_model)  
        top_model = Dense(1024, activation='relu')(top_model)  
        top_model = Dense(1024, activation='relu')(top_model)  
        top_model = Dense(512, activation='relu')(top_model)   
        top_model = Dense(num_classes, activation='softmax')(top_model) 
    
        return top_model 


# Import necessary libraries

In [ ]:

from tensorflow.keras.models import Model
from tensorflow.keras import callbacks
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D


# Edit on VGG-16 Model

In [ ]:
# Define the top layer of the model
def lw(bottom_model, num_classes):
    """Defines the top or head of the model"""
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)  
    top_model = Dense(1024, activation='relu')(top_model) 
    top_model = Dense(1024, activation='relu')(top_model)  
    top_model = Dense(512, activation='relu')(top_model) 
    top_model = Dense(num_classes, activation='softmax')(top_model)  
    return top_model

num_classes = 2

# Loa base VGG16 model without top layers
vgg = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in VGG16
for layer in vgg.layers:
    layer.trainable = False

# Add top layer to the VGG16 model
FC_Head = lw(vgg, num_classes)

# final model by connecting VGG16 model and top layer
model = Model(inputs=vgg.input, outputs=FC_Head)
print(model.summary())



In [ ]:
# Caching and Prefetching (Optimization)
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)

# Define Callbacks
lr_callback = callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.1, patience = 5)
stop_callback = callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 8)

# Compile & Train Model

In [ ]:
EPOCHS=200

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, callbacks = [lr_callback, stop_callback],
                    epochs = EPOCHS, verbose = 1, validation_data=(X_test, y_test), initial_epoch=0)


# Show Train & Validation Accuracy

In [ ]:
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
model.evaluate(test_ds)